<span style="color: red; font-weight: bold">Use the following command to launch this notebook:</span>

```bash
cd marlowe-runtime/
nix run ../marlowe-cli
```

# Various Tests of Marlowe Runtime

## Preliminaries

Record version numbers

In [7]:
marlowe-cli --version

marlowe-cli 0.0.9.0


In [8]:
marlowe --version

Invalid option `--version'

Usage: marlowe [--history-host HOST_NAME] [--history-command-port PORT_NUMBER] 
               [--history-query-port PORT_NUMBER] 
               [--history-sync-port PORT_NUMBER] [--discovery-host HOST_NAME] 
               [--discovery-query-port PORT_NUMBER] [--tx-host HOST_NAME] 
               [--tx-command-port PORT_NUMBER] (COMMAND | COMMAND | COMMAND)

  Command line interface for managing Marlowe smart contracts on Cardano.


: 1

<span style="color: red; font-weight: bold">It looks like <code>marlowe</code> doesn't support <code>--version</code>.</span>

In [9]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [10]:
git rev-parse HEAD

1a4fa82b23df94a4c15d03c9fc30fc535b94aff4


### Setup the faucet.

Set the location of keys.

In [11]:
TREASURY=/extra/iohk/networks/treasury

Set the faucet.

In [12]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In [7]:
#source configure.env
echo "${MAGIC[@]}"

--testnet-magic 2


### Time computations

In [13]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))
#DAY=$((24 * HOUR))

In [14]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1665674005000


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [4]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)

In [5]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [15]:
PARTY_PREFIX="$TREASURY/john-fletcher"
PARTY_SKEY="$PARTY_PREFIX".skey
PARTY_VKEY="$PARTY_PREFIX".vkey

Create the first party's keys, if necessary.

In [16]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY"      \
                              --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2


Fund the address.

In [17]:
marlowe-cli util fund-address --out-file /dev/null \
                              --submit 600 \
                              --lovelace 250000000 \
                              --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
                              "$PARTY_ADDR"

TxId "3736c6f87a02b68f48fbcacf4756913f94540412e166f2b5aa84d3f7674d5145"


#### The Counterparty

In [18]:
COUNTERPARTY_PREFIX="$TREASURY/thomas-kyd"
COUNTERPARTY_SKEY="$COUNTERPARTY_PREFIX".skey
COUNTERPARTY_VKEY="$COUNTERPARTY_PREFIX".vkey

Create the second party's keys, if necessary.

In [19]:
if [[ ! -e "$COUNTERPARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$COUNTERPARTY_SKEY"      \
                              --verification-key-file "$COUNTERPARTY_VKEY"
fi
COUNTERPARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$COUNTERPARTY_VKEY")
echo "$COUNTERPARTY_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


Fund the address.

In [20]:
marlowe-cli util fund-address --out-file /dev/null \
                              --submit 600  \
                              --lovelace 250000000 \
                              --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
                              "$COUNTERPARTY_ADDR"

TxId "caa30cb2710bb44eb13f54338388cfedddde316f18c984cf0634a033bb03d825"


## The Contract

We set the parameters for the ACTUS PAM contract.

In [21]:
MINIMUM_ADA=3000000

FIXED_POINT=1000000
PRINCIPAL=100
INTEREST_RATE=0.02
INTEREST=$(jq -n $PRINCIPAL*$INTEREST_RATE)

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

#LENDING_DEADLINE=$((NOW+12*HOUR))
#REPAYMENT_DEADLINE=$((NOW+24*HOUR))

In [22]:
yaml2json << EOI > runtime-actus-pam.actus
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat runtime-actus-pam.actus

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2023-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2023-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2024-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2022-10-13T00:00:00"}


Create the contract and its initial state.

In [23]:
marlowe-cli template actus --minimum-ada "$MINIMUM_ADA" \
                           --party "$PARTY_ADDR" \
                           --counter-party "$COUNTERPARTY_ADDR" \
                           --actus-terms-file runtime-actus-pam.actus \
                           --out-contract-file runtime-actus-pam-1.contract \
                           --out-state-file    runtime-actus-pam-1.state

View the contract.

In [24]:
json2yaml runtime-actus-pam-1.contract

timeout: 1672531200000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      address: addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2
  then:
    from_account:
      address: addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2
    pay: 100000000
    then:
      timeout: 1704067200000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
        then:
          from_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdm

View the initial state.

In [25]:
json2yaml runtime-actus-pam-1.state

accounts:
- - - address: addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2
    - currency_symbol: ''
      token_name: ''
  - 3000000
boundValues: []
choices: []
minTime: 1


## Create two transactions so that they can be viewed by Marlowe History

### Transaction 1. Create the ontract

First we create a `.marlowe` file that contains the initial information needed to run the contract.

In [26]:
marlowe-cli run initialize --contract-file runtime-actus-pam-1.contract \
                           --state-file    runtime-actus-pam-1.state    \
                           --out-file      runtime-actus-pam-1.marlowe  \
                           --print-stats


Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


Submit the transaction.

In [30]:
TX_1=$(
marlowe-cli run auto-execute --marlowe-out-file runtime-actus-pam-1.marlowe \
                             --required-signer "$PARTY_SKEY" \
                             --change-address "$PARTY_ADDR" \
                             --out-file runtime-actus-pam-1.raw \
                             --print-stats \
                             --submit 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_1 = $TX_1"


Fee: Lovelace 220457
Size: 1133 / 16384 = 6%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX_1 = 3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246


The contract is identified by its first UTxO.

In [31]:
CONTRACT_ID="$TX_1#1"
echo "$CONTRACT_ID"

3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1


### Transaction 2. Party deposits loan amount

In [32]:
marlowe-cli run prepare --marlowe-file runtime-actus-pam-1.marlowe \
                        --out-file     runtime-actus-pam-2.marlowe \
                        --deposit-account "$PARTY_ADDR" \
                        --deposit-party "$PARTY_ADDR" \
                        --deposit-amount "$((PRINCIPAL*FIXED_POINT))" \
                        --invalid-before "$((NOW - 5 * MINUTE))" \
                        --invalid-hereafter "$((NOW + 1 * HOUR))" \
                        --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1665673705000},POSIXTime {getPOSIXTime = 1665677605999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1665673705000},POSIXTime {getPOSIXTime = 1665677605999}), txInputs = [NormalInput (IDeposit "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\"" "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\"" (Token "" "") 100000000)]}

Datum size: 605
Payment 1
  Acccount: "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\""
  Payee: Party "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Ada: Lovelace {getLovelace = 100000000}


Submit the transaction.

In [33]:
TX_2=$(
marlowe-cli run auto-execute --tx-in-marlowe "$TX_1#1" \
                             --marlowe-in-file  runtime-actus-pam-1.marlowe \
                             --marlowe-out-file runtime-actus-pam-2.marlowe \
                             --required-signer "$PARTY_SKEY" \
                             --change-address "$PARTY_ADDR" \
                             --out-file runtime-actus-pam-2.raw \
                             --print-stats \
                             --submit 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_2 = $TX_2"


Fee: Lovelace 1294779
Size: 14464 / 16384 = 88%
Execution units:
  Memory: 6287250 / 14000000 = 44%
  Steps: 1732252213 / 10000000000 = 17%
TX_2 = 4a59aeb4876c8972c9f7125d70a96da37ea061c01430998d9f93108fd17907dc


See if we could have submitted the transaction via Marlowe Runtime.

In [34]:
cardano-cli transaction sign --tx-body-file  runtime-actus-pam-2.raw \
  --signing-key-file "$PARTY_SKEY" --out-file runtime-actus-pam-2.signed

In [36]:
marlowe submit runtime-actus-pam-2.signed

recvMsgFailed
SubmitFailed "TxValidationErrorInMode (ShelleyTxValidationError ShelleyBasedEraBabbage (ApplyTxError [UtxowFailure (FromAlonzoUtxowFail (WrappedShelleyEraFailure (ExtraneousScriptWitnessesUTXOW (fromList [ScriptHash \"6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0\"])))),UtxowFailure (FromAlonzoUtxowFail (NonOutputSupplimentaryDatums (fromList [SafeHash \"94741cba385c77a6bb2feaa5472c4e87fe3f753064710639085127b382a5b828\"]) (fromList [SafeHash \"dab612860b36388b35ab5c6fd205c65d81812450e14e18487bf7e06ed640f4dc\"]))),UtxowFailure (FromAlonzoUtxowFail (ExtraRedeemers [RdmrPtr Spend 1])),UtxowFailure (FromAlonzoUtxowFail (PPViewHashesDontMatch (SJust (SafeHash \"ccb07709628698071b7dcdf587bda7775942b0a29274c4c7c77fdbf5fbf05a75\")) (SJust (SafeHash \"4f555d3c02c9e4d8c5cad721a07805fbfb399765069abf092a168ce95033d76e\")))),UtxowFailure (UtxoFailure (FromAlonzoUtxoFail (ValueNotConservedUTxO (Value 0 (fromList [])) (Value 249779543 (fromList []))))),UtxowFailure (UtxoFailu

: 1

## Contract history commands

### Add the contract to Marlowe History

In [37]:
marlowe add "$CONTRACT_ID"

3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1


### List the contracts managed by Marlowe History

In [38]:
marlowe ls --show-status

3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1 Status: Waiting (SomeMarloweVersion MarloweV1)


### Check removal of the contract from Marlowe History

Remove it.

In [39]:
marlowe rm "$CONTRACT_ID"

3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1


See that it has been removed.

In [40]:
marlowe ls

Add it back.

In [41]:
marlowe add "$CONTRACT_ID"

3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1


In [42]:
marlowe ls

3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1


### List all Marlowe contracts

In [44]:
marlowe ls --all --show-status | wc -l

227


In [43]:
marlowe ls --all --show-status | head -n 20

02811e36c6cdac4721b53f718c4a1406e09ef0d985f9ad6b7fd676769e2f866c#1 Status: not added
045d4ac6f50c57cade58fd697b907ff0418637ba1eadb36392e04d1c5d489b21#1 Status: not added
0552fc00860d35d45c09b4b43740193579adbeea5097cf33b62201332e3095ee#1 Status: not added
05574d9d1935d2a5d2e3cb15e79d46b076e4c81d85bd2e12940ed47b628001be#1 Status: not added
05990678496287262051fc831ff35903e383ffc564b943de76e78394a8ceb4cd#1 Status: not added
061d29872aae0580496aecc8bed3a7a11d743c55d24f6d2c5c03c630cf89e899#1 Status: not added
066727dc5d40a6df33bb88dea3d29a9c1bb9b68493870ef9a04d6cbffe11e21b#1 Status: not added
06718671646f66d1818fce1d2532a3bb2050a42df164c71c3306f9c67ba6f1ab#1 Status: not added
06ddf0488df24a26882abf9546a8d314994b768465246172f8b4492e8a820162#1 Status: not added
072b7513bfae663c993d21977e6b1c2231a6fb389d09595a1e3d4ee517b5a6ed#1 Status: not added
07475c7a0206e0ec98e52c6891bfa5cda2e65b814f08f66f19a7d9958af989ad#1 Status: not added
0b25d59061a5748f0aef14b5a4e8829f0a4940a7aa988325bfd197584de2da58#

### See if there are any failed contracts

In [45]:
marlowe ls --all > all.tmp
marlowe ls --all --show-failed > failed.tmp
diff all.tmp failed.tmp

<span style="color: red; font-weight: bold">I think that <code>--show-failed</code> should <em>only</em> show the failed contracts, not all of them.</span>

### View the basic information about the contract

In [46]:
marlowe log "$CONTRACT_ID"

transaction 3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246 (creation)
ContractId:      3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1
SlotNo:          5671324
BlockNo:         261181
BlockId:         4fff144e296fc37bbe73b394340d963f7ac7cfbd96e57c75b72399006c811f97
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1



transaction 4a59aeb4876c8972c9f7125d70a96da37ea061c01430998d9f93108fd17907dc
ContractId: 3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1
SlotNo:     5671346
BlockNo:    261183
BlockId:    4f99a3ce9bf8850a6bc9c460fb213e66331d6c79105b87127d49c0148d1cadbb
Inputs:     [NormalInput (IDeposit "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\"" "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\"" (Token "" "") 100000000)]




<span style="color: red; font-weight: bold">I think that <code>marlowe log</code> should print something parseable like YAML.</span>

### View the contract itself

In [47]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246 (creation)
ContractId:      3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1
SlotNo:          5671324
BlockNo:         261181
BlockId:         4fff144e296fc37bbe73b394340d963f7ac7cfbd96e57c75b72399006c811f97
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2" Address "addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

<span style="color: red; font-weight: bold">I think that <code>marlowe log --show-contract</code> should print something parseable like YAML.</span>

## Finish running the contract

### Transaction 3. Counterparty repays the loan's interest

In [48]:
marlowe-cli run prepare --marlowe-file runtime-actus-pam-2.marlowe \
                        --out-file     runtime-actus-pam-3.marlowe \
                        --deposit-account "$COUNTERPARTY_ADDR" \
                        --deposit-party "$COUNTERPARTY_ADDR" \
                        --deposit-amount "$((INTEREST*FIXED_POINT))" \
                        --invalid-before "$((NOW-5*MINUTE))" \
                        --invalid-hereafter "$((NOW+1*HOUR))" \
                        --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1665673705000},POSIXTime {getPOSIXTime = 1665677605999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1665673705000},POSIXTime {getPOSIXTime = 1665677605999}), txInputs = [NormalInput (IDeposit "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" (Token "" "") 2000000)]}

Datum size: 349
Payment 1
  Acccount: "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Payee: Party "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\""
  Ada: Lovelace {getLovelace = 2000000}


In [49]:
TX_3=$(
marlowe-cli run auto-execute --tx-in-marlowe "$TX_2#1" \
                             --marlowe-in-file  runtime-actus-pam-2.marlowe \
                             --marlowe-out-file runtime-actus-pam-3.marlowe \
                             --required-signer "$COUNTERPARTY_SKEY" \
                             --change-address "$COUNTERPARTY_ADDR" \
                             --out-file runtime-actus-pam-3.raw \
                             --print-stats \
                             --submit 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_3 = $TX_3"


Fee: Lovelace 1236221
Size: 13952 / 16384 = 85%
Execution units:
  Memory: 5855282 / 14000000 = 41%
  Steps: 1578216249 / 10000000000 = 15%
TX_3 = 5c69673b1e608df8e7eed605853e27dc7932dfba2a378740b0ac77c529bfc56c


### Transaction 4. Counterparty repays the loan's principal

In [50]:
marlowe-cli run prepare --marlowe-file runtime-actus-pam-3.marlowe \
                        --out-file     runtime-actus-pam-4.marlowe \
                        --deposit-account "$COUNTERPARTY_ADDR" \
                        --deposit-party "$COUNTERPARTY_ADDR" \
                        --deposit-amount "$((PRINCIPAL*FIXED_POINT))" \
                        --invalid-before "$((NOW-5*MINUTE))" \
                        --invalid-hereafter "$((NOW+4*HOUR))" \
                        --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1665673705000},POSIXTime {getPOSIXTime = 1665688405999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1665673705000},POSIXTime {getPOSIXTime = 1665688405999}), txInputs = [NormalInput (IDeposit "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" (Token "" "") 100000000)]}

Datum size: 30
Payment 1
  Acccount: "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Payee: Party "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\""
  Ada: Lovelace {getLovelace = 100000000}
Payment 2
  Acccount: "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\""
  Payee: Party "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\""
  Ada: Lovelace {getLovelace = 3000000}


In [52]:
TX_4=$(
marlowe-cli run auto-execute --tx-in-marlowe "$TX_3#1" \
                             --marlowe-in-file  runtime-actus-pam-3.marlowe \
                             --marlowe-out-file runtime-actus-pam-4.marlowe \
                             --required-signer "$COUNTERPARTY_SKEY" \
                             --change-address "$COUNTERPARTY_ADDR" \
                             --out-file runtime-actus-pam-4.raw \
                             --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_4 = $TX_4"


Fee: Lovelace 1076839
Size: 13273 / 16384 = 81%
Execution units:
  Memory: 4184150 / 14000000 = 29%
  Steps: 1119389602 / 10000000000 = 11%
TX_4 = 980da6aeb75b655cf41000c527e7e61e9346355c91e76762c921957c23a01350


In [55]:
cardano-cli transaction sign --tx-body-file  runtime-actus-pam-4.raw \
  --signing-key-file "$COUNTERPARTY_SKEY" --out-file runtime-actus-pam-4.signed

In [56]:
marlowe submit runtime-actus-pam-4.signed

BlockHeader {slotNo = SlotNo {unSlotNo = 5672154}, headerHash = "ae1c1002674597d350a7abe19143a193bf53e1a4458f7db272ba14b1deb917af", blockNo = BlockNo {unBlockNo = 261218}}


The contract has closed.

In [58]:
jq '.tx.contract' runtime-actus-pam-4.marlowe | json2yaml

close
...


<span style="color: red; font-weight: bold">The contract has closed, but it looks like Marlowe History thinks it is still open:</span>

In [57]:
marlowe ls --show-status

3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1 Status: Waiting (SomeMarloweVersion MarloweV1)


In [59]:
marlowe log "$CONTRACT_ID"

transaction 3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246 (creation)
ContractId:      3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1
SlotNo:          5671324
BlockNo:         261181
BlockId:         4fff144e296fc37bbe73b394340d963f7ac7cfbd96e57c75b72399006c811f97
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1



transaction 4a59aeb4876c8972c9f7125d70a96da37ea061c01430998d9f93108fd17907dc
ContractId: 3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1
SlotNo:     5671346
BlockNo:    261183
BlockId:    4f99a3ce9bf8850a6bc9c460fb213e66331d6c79105b87127d49c0148d1cadbb
Inputs:     [NormalInput (IDeposit "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\"" "\"addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2\"" (Token "" "") 100000000)]


transaction 5c69673b1e608df8e7eed605853e27dc7932dfba2a378740b0ac77c529bfc56c
ContractId: 3112b7de5b84bd2af6e57a4d2cbc78

In [60]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246 (creation)
ContractId:      3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1
SlotNo:          5671324
BlockNo:         261181
BlockId:         4fff144e296fc37bbe73b394340d963f7ac7cfbd96e57c75b72399006c811f97
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2" Address "addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vr9dd8p3svssr8k2g5kd2syq4ad40nz9370tclufhv7xsjsd8vqv2"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

## Additional testing

<span style="color: red; font-weight: bold">We still have other commands to test:</span>

* apply
* advance
* deposit
* choose
* notify
* create
* withdraw

In [46]:
marlowe --help

Usage: marlowe [--history-host HOST_NAME] [--history-command-port PORT_NUMBER] 
               [--history-query-port PORT_NUMBER] 
               [--history-sync-port PORT_NUMBER] [--discovery-host HOST_NAME] 
               [--discovery-query-port PORT_NUMBER] [--tx-host HOST_NAME] 
               [--tx-command-port PORT_NUMBER] (COMMAND | COMMAND | COMMAND)

  Command line interface for managing Marlowe smart contracts on Cardano.

Available options:
  -h,--help                Show this help text
  --history-host HOST_NAME The hostname of the Marlowe Runtime history server.
                           Can be set as the environment variable
                           MARLOWE_RT_HISTORY_HOST (default: "127.0.0.1")
  --history-command-port PORT_NUMBER
                           The port number of the history server's job API. Can
                           be set as the environment variable
                           MARLOWE_RT_HISTORY_COMMAND_PORT (default: 23717)
  --history-query-port

In [61]:
marlowe rm "$CONTRACT_ID"

3112b7de5b84bd2af6e57a4d2cbc78b1a693e2981ac2b097a248c320f3eaf246#1


In [62]:
marlowe ls

In [71]:
marlowe create --core-file runtime-actus-pam-1.contract \
               --change-address "$PARTY_ADDR" \
               --min-utxo 3000000 \
               --manual-sign tx.raw

CreateFailed (CreateConstraintError (CoinSelectionFailed "Insufficient lovelace available for coin selection: valueFromList [(AdaAssetId,2857910715)] required, but valueFromList [(AdaAssetId,2849260111)] available."))


: 1

In [ ]:
marlowe create --help

In [66]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
48771d4de5928a511ae098913a4a278d6ac97d0667510e63483fb1f42e6d40f6     2        2000000 lovelace + TxOutDatumNone
4a59aeb4876c8972c9f7125d70a96da37ea061c01430998d9f93108fd17907dc     0        145484764 lovelace + TxOutDatumNone
5c69673b1e608df8e7eed605853e27dc7932dfba2a378740b0ac77c529bfc56c     2        2000000 lovelace + TxOutDatumNone
7d23ded10f5710ff03cb1b32f89f43263267f593857b88a612442a3e627111de     2        2000000 lovelace + TxOutDatumNone
881e6526b9799bd95f8f951fb6b75d30c1678193b293e243c5b84fc5f414fa82     0        148708414 lovelace + TxOutDatumNone
8cc21c3f3eb029ce95732429c77adfb472f26224a56a3fc5ec336a3964bd6383     1        103000000 lovelace + TxOutDatumNone
980da6aeb75b655cf41000c527e7e61e9346355c91e76762c921957c23a01350     1        103000000 lovelace + TxOutDatumNone
f298ef07ad63215ae0e286a9d00bbbcbffe5

In [68]:
marlowe-cli util clean --required-signer "$PARTY_SKEY" --change-address "$PARTY_ADDR" --out-file /dev/null --submit 600

TxId "e4795f6dac4d47c455007a606d9f3efe40a9099ae072ef4fc9158ff625859394"


In [74]:
echo -n Marlowe | basenc --base16

4D61726C6F7765


In [77]:
echo -n 4D61726C6F7765 | tr '[:upper:]' '[:lower:]'

4d61726c6f7765

In [76]:
echo -n 4D61726C6F7765 | tr '[:upper:]' '[:lower:]' | basenc --decode --base16

basenc: invalid input


: 1

In [69]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
e4795f6dac4d47c455007a606d9f3efe40a9099ae072ef4fc9158ff625859394     0        751243540 lovelace + TxOutDatumNone


In [72]:
marlowe-cli util select --lovelace-only 1000000 "$PARTY_ADDR"

TxIn "e4795f6dac4d47c455007a606d9f3efe40a9099ae072ef4fc9158ff625859394" (TxIx 0)


In [73]:
marlowe-cli util select

Missing: 
(--lovelace-only LOVELACE | --asset-only CURRENCY_SYMBOL.TOKEN_NAME | --all)
ADDRESS

Usage: marlowe-cli util select [--testnet-magic INTEGER] 
                               [--socket-path SOCKET_FILE] 
                               (--lovelace-only LOVELACE | 
                                 --asset-only CURRENCY_SYMBOL.TOKEN_NAME | 
                                 --all) ADDRESS

  Select UTxO by asset.


: 1